In [ ]:
# @title 🔨 1. 安装依赖
import subprocess
import sys
import os

def run_cmd(cmd, cwd=None):
    result = subprocess.run(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, cwd=cwd)
    if result.returncode != 0:
        print(f"❌ 命令执行出错: {cmd}")
        print(result.stderr.decode('utf-8'))
        sys.exit(1)

print('⏳ 正在安装依赖...')

try:
    # 安装 Node.js 20
    run_cmd('curl -fsSL https://deb.nodesource.com/setup_20.x | bash -')
    run_cmd('apt-get install -y nodejs')

    # 克隆 GitHub 项目
    run_cmd('git clone https://github.com/dreamhartley/JimiHub.git')

    # 进入项目目录
    os.chdir('JimiHub')

    # 使用 npm 安装项目依赖
    run_cmd('npm install')

    print('✅ 依赖安装完成！')
except Exception as e:
    print('❌ 安装过程中出现了错误：', e)


In [ ]:
# @title ⚙️ 2. 配置环境变量
%cd /content/JimiHub
# @markdown 请在下面的表单中填入您的配置信息。
# @markdown ---

# @markdown ### **必填项**
# @markdown **1. 管理员密码 (`ADMIN_PASSWORD`)**
# @markdown 后台管理面板的登录密码。
admin_password = ""  #@param {type:"string"}

# @markdown **2. GitHub 项目 (`GITHUB_PROJECT`)**
# @markdown 您的 GitHub 项目路径，格式为 `用户名/仓库名`。
github_project = ""  #@param {type:"string"}

# @markdown **3. GitHub PAT (`GITHUB_PROJECT_PAT`)**
# @markdown 用于访问 GitHub 仓库的个人访问令牌 (Personal Access Token)。请确保它具有读写仓库内容的权限。
github_pat = ""  #@param {type:"string"}

# @markdown ---
# @markdown ### **可选项**
# @markdown **4. 加密密钥 (`GITHUB_ENCRYPT_KEY`)**
# @markdown （可选）一个32位长度的字符串，用于加密敏感数据，对于已加密的数据库必须使用和之前相同的密钥。如果留空，则不启用加密。
github_encrypt_key = ""  #@param {type:"string"}


# --- 后续逻辑 (无需修改) ---
# 检查必填项是否已填写
if not all([admin_password, github_project, github_pat]):
    print("❌ 错误：管理员密码、GitHub 项目和 PAT 均为必填项，请填写后再运行！")
else:
    # 使用从表单获取的变量创建 .env 文件内容
    # .strip() 用于移除开头和结尾可能存在的空白
    env_content = f"""
ADMIN_PASSWORD={admin_password}
GITHUB_PROJECT={github_project}
GITHUB_PROJECT_PAT={github_pat}
GITHUB_ENCRYPT_KEY={github_encrypt_key}
""".strip()

    # 将内容写入 .env 文件
    with open(".env", "w") as f:
        f.write(env_content)

    print("✅ .env 文件已成功创建并写入以下内容：")
    # 打印文件内容以供核对
    !cat .env



In [ ]:
# @title 🚀 3. 启动面板
# 下载 Cloudflare Tunnel
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O cloudflared
!chmod +x cloudflared

import subprocess
import time
from IPython.display import display, HTML
import re

def start_cloudflare_tunnel(log_path='tunnel.log', retries=5, wait_sec=5):
    """启动 Cloudflare Tunnel 并返回访问 URL，失败时自动重试。"""
    tunnel_cmd = './cloudflared tunnel --url http://127.0.0.1:3000 > {} 2>&1 &'.format(log_path)
    for attempt in range(1, retries + 1):
        # 启动隧道
        subprocess.Popen(tunnel_cmd, shell=True)
        print(f"第 {attempt} 次尝试启动 Cloudflare Tunnel...")
        time.sleep(wait_sec)
        # 提取 URL
        tunnel_url = None
        try:
            with open(log_path, 'r', encoding='utf-8', errors='ignore') as f:
                for line in f:
                    match = re.search(r'https://[a-zA-Z0-9-]+\.trycloudflare\.com', line)
                    if match:
                        tunnel_url = match.group(0)
                        break
        except Exception as e:
            print(f"读取日志文件异常: {e}")
        if tunnel_url:
            return tunnel_url
        else:
            print("未获取到 URL，准备重试...")
            # 杀掉可能遗留的 cloudflared 进程
            subprocess.run("pkill -f cloudflared", shell=True)
            time.sleep(1)
    return None

tunnel_url = start_cloudflare_tunnel()

if tunnel_url:
    # 美化的 HTML 面板
    panel_html = f"""
    <div style="
        border: 2px solid #2196F3;
        border-radius: 14px;
        background: linear-gradient(90deg,#f5f8fa 60%,#e3f2fd 100%);
        padding: 24px 32px;
        box-shadow: 0 2px 12px rgba(33,150,243,0.07);
        margin: 18px 0;
        font-family: 'Segoe UI',Arial,sans-serif;
        font-size: 1.15em;
        color: #222;
        width: fit-content;
        max-width: 90vw;
      ">
      <div style="font-size:1.3em;font-weight:bold;color:#1976D2;margin-bottom:8px;">
        ✅ Cloudflare Tunnel 已启动
      </div>
      <div>
        <span style="font-weight:bold;">访问地址：</span>
        <a href="{tunnel_url}" target="_blank" style="color:#1976D2;text-decoration:underline;">{tunnel_url}</a>
      </div>
      <div style="margin-top:6px;">
        <span style="font-weight:bold;">API端点：</span>
        <a href="{tunnel_url}/v1" target="_blank" style="color:#388E3C;text-decoration:underline;">{tunnel_url}/v1</a>
      </div>
    </div>
    """
    display(HTML(panel_html))
else:
    error_html = """
    <div style="
        border:2px solid #E53935;
        border-radius:10px;
        background:#FFF3F3;
        padding:18px 24px;
        color:#B71C1C;
        font-weight:bold;
        font-size:1.15em;
        width: fit-content;
        max-width: 90vw;
        margin: 18px 0;">
      ❌ <span>未能获取 Cloudflare Tunnel 的临时 URL，请检查 <code>tunnel.log</code> 或稍后重试。</span>
    </div>
    """
    display(HTML(error_html))

# 启动 Node.js 项目
print("\n正在后台启动 Node.js 项目 (npm start)...")
!npm start
